# MWC - Barcelona

In [ ]:
# relevant imports 

import requests as re
import json
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import time


In [ ]:
#define headers
headers = {
    'Accept': '*/*',
    'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
    'Cache-Control': 'no-cache',
    'Connection': 'keep-alive',
    'Origin': 'https://www.mwcbarcelona.com',
    'Pragma': 'no-cache',
    'Referer': 'https://www.mwcbarcelona.com/exhibitors',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'cross-site',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'content-type': 'application/x-www-form-urlencoded',
    'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}

In [ ]:
# Define data
data = {
    "requests":[
        {
            "indexName":"exhibitors-default",
            "params": ""
        }
    ]
}

In [ ]:
# Scrape urls of the event page for each partecipants page by page

page = 0
results = True
result_urls = []

while results: 
    data["requests"][0]["params"] = "facets=%5B%22building%22%2C%22interests.label%22%2C%22letter%22%5D&highlightPostTag=%3C%2Fais-highlight-0000000000%3E&highlightPreTag=%3Cais-highlight-0000000000%3E&hitsPerPage=24&maxValuesPerFacet=500&page="+str(page)+"&query=&tagFilters="
    
    res = re.post(
        'https://8vvb6vr33k-dsn.algolia.net/1/indexes/*/queries?x-algolia-agent=Algolia%20for%20JavaScript%20(4.14.3)%3B%20Browser%20(lite)%3B%20JS%20Helper%20(3.11.3)%3B%20react%20(17.0.2)%3B%20react-instantsearch%20(6.34.0)&x-algolia-api-key=00422c3d9f3484bccfae011262fcf49a&x-algolia-application-id=8VVB6VR33K',
        headers=headers,
        data=json.dumps(data)
    ).json()['results']
    
    print(res)
    if len(res[0]['hits']) > 0:
        res = res[0]['hits']
        for r in res:
            result_urls.append(r['url'])
    else:
        break

    page = page+1

In [ ]:
result_urls

In [ ]:
#create dataframe with url of event page for each partecipant
df = pd.DataFrame(result_urls)

In [ ]:
df

,0
0,https://www.mwcbarcelona.com/exhibitors/airbus...
1,https://www.mwcbarcelona.com/exhibitors/cariad-se
2,https://www.mwcbarcelona.com/exhibitors/orange
3,https://www.mwcbarcelona.com/exhibitors/ribbon
4,https://www.mwcbarcelona.com/exhibitors/360-pu...
...,...
1995,https://www.mwcbarcelona.com/exhibitors/triali...
1996,https://www.mwcbarcelona.com/exhibitors/trinam...
1997,https://www.mwcbarcelona.com/exhibitors/triple...
1998,https://www.mwcbarcelona.com/exhibitors/trivana


In [ ]:
# export dataframe to csv
df.to_csv('mwc_urls.csv', index=False)

# Go into each link and get company name + check if there is a Web button, if it is get the href link to it

In [3]:
# import dataframe
df = pd.read_csv('mwc_urls.csv')

In [4]:
df 

,0
0,https://www.mwcbarcelona.com/exhibitors/airbus...
1,https://www.mwcbarcelona.com/exhibitors/cariad-se
2,https://www.mwcbarcelona.com/exhibitors/orange
3,https://www.mwcbarcelona.com/exhibitors/ribbon
4,https://www.mwcbarcelona.com/exhibitors/360-pu...
...,...
1995,https://www.mwcbarcelona.com/exhibitors/triali...
1996,https://www.mwcbarcelona.com/exhibitors/trinam...
1997,https://www.mwcbarcelona.com/exhibitors/triple...
1998,https://www.mwcbarcelona.com/exhibitors/trivana


In [5]:
# name the unnamed column
df.columns = ['MWC_Link']

In [6]:
# create a backup of the dataframe
df_backup=df.copy()

In [8]:
# define headers
header={
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "accept-language": "en-GB,en-US;q=0.9,en;q=0.8",
    "cache-control": "no-cache",
    "pragma": "no-cache",
    "sec-ch-ua": "\"Not?A_Brand\";v=\"8\", \"Chromium\";v=\"108\", \"Google Chrome\";v=\"108\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"macOS\"",
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "same-origin",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1",

    #"referrer": "https://fccid.io/",
    "referrerPolicy": "strict-origin-when-cross-origin",
   
    "method": "GET",
    "mode": "cors",
    "credentials": "include"
  }

In [9]:
# create function to scrape the company website from each partecipant page in the event website
def get_ex_website(url_to_use):
    
    r = re.get(url_to_use,headers=header)
    time.sleep(0.2)

    soup = BeautifulSoup(r.text)
    aside = soup.find(lambda tag: tag.name=='aside') 
    #print(aside)
    try:
        a= aside.find_all('a')
        k=a[-1]
        name=k['href'] # identify link to company website
        return name
    except:
        return 'no website (yet)'

In [10]:
# apply function to dataframe

df['domain'] = df.MWC_Link.apply(lambda x: get_ex_website(x))

In [11]:
# extract the company name from the company url
def get_company_name(og_link):
    cname=og_link.split("s/",1)[1] # split the url to get the company name
    cname=cname.replace('-',' ')
    return cname

In [12]:
# apply function to dataframe

df['company name'] = df.MWC_Link.apply(lambda x: get_company_name(x))

In [13]:
df

,MWC_Link,domain,company name
0,https://www.mwcbarcelona.com/exhibitors/airbus...,https://aaltohaps.com,airbus haps connectivity solutions ltd
1,https://www.mwcbarcelona.com/exhibitors/cariad-se,https://cariad.technology,cariad se
2,https://www.mwcbarcelona.com/exhibitors/orange,https://www.orange.com/en,orange
3,https://www.mwcbarcelona.com/exhibitors/ribbon,https://ribboncommunications.com/company/media...,ribbon
4,https://www.mwcbarcelona.com/exhibitors/360-pu...,https://pulsus.mobi/en/,360 pulsus
...,...,...,...
1995,https://www.mwcbarcelona.com/exhibitors/triali...,https://www.trialing.org,trialing health sl
1996,https://www.mwcbarcelona.com/exhibitors/trinam...,https://www.trinamiXsensing.com,trinamix gmbh
1997,https://www.mwcbarcelona.com/exhibitors/triple...,https://triple-innovations.com/,triple innovations ofca
1998,https://www.mwcbarcelona.com/exhibitors/trivana,no website (yet),trivana


In [14]:
# export dataframe to csv
df.to_csv('mwc_websites_available.csv', index=False)

# Check if company is in Hubspot  by domain or name

In [1]:
df=backup_df

In [ ]:
key= 'key' #removed for privacy reasons

In [ ]:
# clean domain to get its based form (e.g. www.google.com -> google.com), as saved in HubSpot
def _domain_cleaner(link):
  domain = link.replace('https://','')
  domain = domain.replace('http://','')
  domain = domain.replace('www.','')
  domain = domain.split('/')[0]
  return domain

In [ ]:
# This one is for HubSpot -> prop is HubSpot property value used for comparison
def domain_in_HS(prop, value):
    if value !='no website (yet) ': # if the company has a website
        value = _domain_cleaner(value)
    #print(value)
    uri = "https://api.hubapi.com/crm/v3/objects/companies/search"
    headers = {'authorization': key,'content-type': 'application/json'}
    body = {"filterGroups": [ { "filters": [ { "value": value, "propertyName": prop, "operator": "EQ" } ] }]} 
    res = re.post(uri, headers=headers, data=json.dumps(body)).json()
    time.sleep(0.2)
    #f=res['results'][0]
    #print(f['id'])

    try:
        if len(res['results']) > 0:
            f=res['results'][0]
            return f['id']
        else:
            return 'no id'
    except:
        return 'no id'

In [ ]:
# try to find the company domain in HubSpot
df['domain_HS'] = df.domain.apply(lambda x: domain_in_HS('domain',x))

In [ ]:
df

,MWC_Link,domain,company name,domain_HS,name_HS
0,https://www.mwcbarcelona.com/exhibitors/airbus...,https://aaltohaps.com,airbus haps connectivity solutions ltd,no id,no id
1,https://www.mwcbarcelona.com/exhibitors/cariad-se,https://cariad.technology,cariad se,no id,no id
2,https://www.mwcbarcelona.com/exhibitors/networ...,https://www.networkme.io,network me vieira manteigas lda,no id,no id
3,https://www.mwcbarcelona.com/exhibitors/01synergy,no website (yet),01synergy,no id,no id
4,https://www.mwcbarcelona.com/exhibitors/1stzoom,no website (yet),1stzoom,no id,no id
...,...,...,...,...,...
903,https://www.mwcbarcelona.com/exhibitors/transatel,https://www.transatel.com,transatel,11401582968,11401582968
904,https://www.mwcbarcelona.com/exhibitors/transc...,no website (yet),transcom instruments co ltd,no id,no id
905,https://www.mwcbarcelona.com/exhibitors/trasna...,no website (yet),trasna solutions gmbh,no id,no id
906,https://www.mwcbarcelona.com/exhibitors/trend-...,no website (yet),trend micro incorporated,no id,no id


In [ ]:
def name_in_HS(prop, value):
    #print(value)
    uri = "https://api.hubapi.com/crm/v3/objects/companies/search"
    headers = {'authorization': key,'content-type': 'application/json'}
    body = {"filterGroups": [ { "filters": [ { "value": value, "propertyName": prop, "operator": "EQ" } ] }]}
    res = re.post(uri, headers=headers, data=json.dumps(body)).json()
    time.sleep(0.2)
    try:
        if len(res['results']) > 0:
            f=res['results'][0]
            return f['id']
        else:
            return 'no id'
    except:
        return 'no id'

In [ ]:
# try to find the company name in HubSpot

df['name_HS'] = df['company name'].apply(lambda x: name_in_HS('name',x))

In [ ]:
df

,MWC_Link,domain,company name,domain_HS,name_HS
0,https://www.mwcbarcelona.com/exhibitors/airbus...,https://aaltohaps.com,airbus haps connectivity solutions ltd,no id,no id
1,https://www.mwcbarcelona.com/exhibitors/cariad-se,https://cariad.technology,cariad se,no id,no id
2,https://www.mwcbarcelona.com/exhibitors/networ...,https://www.networkme.io,network me vieira manteigas lda,no id,no id
3,https://www.mwcbarcelona.com/exhibitors/01synergy,no website (yet),01synergy,no id,no id
4,https://www.mwcbarcelona.com/exhibitors/1stzoom,no website (yet),1stzoom,no id,no id
...,...,...,...,...,...
903,https://www.mwcbarcelona.com/exhibitors/transatel,https://www.transatel.com,transatel,11401582968,11401582968
904,https://www.mwcbarcelona.com/exhibitors/transc...,no website (yet),transcom instruments co ltd,no id,no id
905,https://www.mwcbarcelona.com/exhibitors/trasna...,no website (yet),trasna solutions gmbh,no id,no id
906,https://www.mwcbarcelona.com/exhibitors/trend-...,no website (yet),trend micro incorporated,no id,no id


In [ ]:
df.to_csv('mwc_companies_hs_id.csv', index=False)

# Add company property

In [ ]:
df = pd.read_csv("mwc_companies_hs_id.csv")

In [ ]:
df

,MWC_Link,domain,company name,domain_HS,name_HS
0,https://www.mwcbarcelona.com/exhibitors/airbus...,no website (yet),airbus haps connectivity solutions ltd,no id,no id
1,https://www.mwcbarcelona.com/exhibitors/cariad-se,no website (yet),cariad se,no id,no id
2,https://www.mwcbarcelona.com/exhibitors/orange,https://www.orange.com/en,orange,6627256582,6627256582
3,https://www.mwcbarcelona.com/exhibitors/ribbon,https://ribboncommunications.com/company/media...,ribbon,no id,no id
4,https://www.mwcbarcelona.com/exhibitors/360-pu...,https://pulsus.mobi/en/,360 pulsus,no id,no id
...,...,...,...,...,...
1995,https://www.mwcbarcelona.com/exhibitors/triali...,https://www.trialing.org,trialing health sl,no id,no id
1996,https://www.mwcbarcelona.com/exhibitors/trinam...,https://www.trinamiXsensing.com,trinamix gmbh,no id,no id
1997,https://www.mwcbarcelona.com/exhibitors/triple...,https://triple-innovations.com/,triple innovations ofca,no id,no id
1998,https://www.mwcbarcelona.com/exhibitors/trivana,no website (yet),trivana,no id,no id


In [ ]:
# export companies with no website to perform manual research
df = df[df['domain'] == "no website (yet)"]

In [ ]:
df.to_csv('un_matched_urls.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=97a13265-406c-4589-bbe2-27e9eaaf3560' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>